## Задание

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
    - Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
    - Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

---

In [1]:
#!pip install xlrd
#!pip install stop_words

In [2]:
import os
import pandas as pd

In [3]:
df = pd.read_excel('отзывы за лето.xls')

In [4]:
df = df[['Rating','Content']]
df.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [5]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [6]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

---

In [7]:
!pip install wget

In [8]:
max_words = 1000
max_sentence_len = 100

In [9]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py
from  rus_preprocessing_udpipe import process

import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
from tqdm import tqdm

tqdm.pandas()

In [10]:
df.head(5)

,Rating,Content
0,1,It just works!
1,1,В целом удобноное приложение...из минусов хотя...
2,1,Отлично все
3,1,Стал зависать на 1% работы антивируса. Дальше ...
4,1,"Очень удобно, работает быстро."


In [11]:
def replase_words(text,dict_ = emoticon_dict): 
    text = str(text)
    output = ''
    for word in text.split(' '):
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

In [12]:
df['Content'] = df['Content'].apply(replase_words)

In [13]:
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)


In [14]:
res = 'очень странный текст о котором мы хотим поговорить'
print(res)
output = process(process_pipeline, text=res)
print(" ".join(output))

очень странный текст о котором мы хотим поговорить
очень_ADV странный_ADJ текст_NOUN о_ADP который_PRON мы_PRON хотеть_VERB говорить_VERB


In [15]:
df['Content'] = df['Content'].progress_apply(lambda x:" ".join(process(process_pipeline, text = x)))

100%|████████████████████████████████████| 20659/20659 [00:57<00:00, 358.84it/s]


In [16]:
df_train, df_test = train_test_split(df,test_size = 0.3)

In [17]:
train_corpus = " ".join(df_train["Content"])

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ilya.ivolgin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
tokens_filtered = [word for word in tokens]
tokens_filtered

['привет_VERB',
 'весь_DET',
 'приложение_NOUN',
 'очень_ADV',
 'удобный_ADJ',
 'и_CCONJ',
 'быстро_ADV',
 'все_PRON',
 'оплачивать_VERB',
 'и_CCONJ',
 'ни_PART',
 'куда_ADV',
 'не_PART',
 'надо_ADV',
 'ходить_VERB',
 'любить_VERB',
 'я_PRON',
 'любить_VERB',
 'вы_PRON',
 'просто_ADJ',
 'и_CCONJ',
 'удобный_ADJ',
 'все_PRON',
 'устраивать_VERB',
 'спасибо_PROPN',
 'все_PRON',
 'за_ADP',
 'отличный_ADJ',
 'работа_NOUN',
 'очень_ADV',
 'хороший_ADJ',
 'и_CCONJ',
 'нужный_ADJ',
 'программа_NOUN',
 'отличный_ADV',
 'работать_VERB',
 'настоящий_ADJ',
 'верный_ADJ',
 'помошник_NOUN',
 'очень_ADV',
 'нужный_ADJ',
 'приложение_NOUN',
 'разбираться_VERB',
 'молодец_NOUN',
 'хороший_ADJ',
 'приложение_NOUN',
 'все_PRON',
 'работать_VERB',
 'очень_ADV',
 'быстро_ADV',
 'и_CCONJ',
 'удобный_ADV',
 'я_PRON',
 'нравиться_VERB',
 'весь_PRON',
 'удобный_ADJ',
 'и_CCONJ',
 'просто_PART',
 'хороший_ADJ',
 'приложение_NOUN',
 'иногда_ADV',
 'висеть_VERB',
 'очень_ADV',
 'удобный_ADJ',
 'молодцы☺_NOUN',
 

In [20]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [21]:
tokens_filtered_top[:10]

['приложение_NOUN',
 'не_PART',
 'удобный_ADJ',
 'и_CCONJ',
 'очень_ADV',
 'все_PRON',
 'в_ADP',
 'я_PRON',
 'на_ADP',
 'работать_VERB']

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'приложение_NOUN': 1,
 'не_PART': 2,
 'удобный_ADJ': 3,
 'и_CCONJ': 4,
 'очень_ADV': 5,
 'все_PRON': 6,
 'в_ADP': 7,
 'я_PRON': 8,
 'на_ADP': 9,
 'работать_VERB': 10,
 'с_ADP': 11,
 'отличный_ADJ': 12,
 'хороший_ADJ': 13,
 ',': 14,
 'нравиться_VERB': 15,
 'раз_NOUN': 16,
 'по_ADP': 17,
 'но_CCONJ': 18,
 'телефон_NOUN': 19,
 'что_SCONJ': 20,
 'после_ADP': 21,
 'мочь_VERB': 22,
 'спасибо_PROPN': 23,
 'а_CCONJ': 24,
 'что_PRON': 25,
 'x_NUM': 26,
 'это_PRON': 27,
 'весь_DET': 28,
 'обновление_NOUN': 29,
 'супер_NOUN': 30,
 'при_ADP': 31,
 'пользоваться_VERB': 32,
 'так_ADV': 33,
 'за_ADP': 34,
 'быть_AUX': 35,
 'и_PART': 36,
 'весь_PRON': 37,
 'только_PART': 38,
 'банк_NOUN': 39,
 'нет_VERB': 40,
 'быть_VERB': 41,
 'делать_VERB': 42,
 'пароль_NOUN': 43,
 'быстро_ADV': 44,
 'антивирус_NOUN': 45,
 'устраивать_VERB': 46,
 'вход_NOUN': 47,
 'вы_PRON': 48,
 'заходить_VERB': 49,
 'карта_NOUN': 50,
 'у_ADP': 51,
 'прошивка_NOUN': 52,
 'он_PRON': 53,
 'этот_DET': 54,
 'как_SCONJ': 55,
 'для_ADP'

In [23]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [24]:
x_train = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_sentence_len) for text in df_test["Content"]], dtype=np.int32)

---

### Обучение сети

In [25]:
import numpy as np
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D,GlobalMaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.callbacks import EarlyStopping  
from sklearn.metrics import accuracy_score 

In [26]:
num_classes = 2
epochs = 20
batch_size = 512
print_batch_n = 100

In [30]:
#!wget "http://vectors.nlpl.eu/repository/20/180.zip"
#!unzip "180.zip"

Archive:  180.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [28]:
import sys
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

In [32]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary = True)
pretrained_weights = word_model.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print(f'vocab_size: {vocab_size}, emdedding_size: {emdedding_size}')

2022-07-16 22:51:56,302 : INFO : loading projection weights from model.bin
2022-07-16 22:51:57,916 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (189193, 300) matrix of type float32 from model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-07-16T22:51:57.916322', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


vocab_size: 189193, emdedding_size: 300


In [33]:
train_count = len(df_train)
test_count = len(df_test)

In [34]:
df_train.head()

,Rating,Content
402,1,привет_VERB весь_DET приложение_NOUN очень_ADV...
17556,1,любить_VERB
11845,1,я_PRON любить_VERB вы_PRON
16240,1,просто_ADJ и_CCONJ удобный_ADJ
19908,1,все_PRON устраивать_VERB


In [35]:
def word2idx(word):
    return word_model.key_to_index[word]
def idx2word(idx):
    return word_model.index_to_key[idx]


train_y = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
train_x = np.zeros([train_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_train['Content']):
    tokens = word_tokenize(sentence)
    for t, word in enumerate(tokens):
        try:
            train_x[i, t] = word2idx(word)
        except:
            continue

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (14461, 100)
train_y shape: (14461, 2)


In [36]:
x = df_test['Content']
y = df_test['Rating']

test_y = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)
test_x = np.zeros([test_count, max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(df_test['Content']):
    tokens = word_tokenize(sentence)
    for t, word in enumerate(tokens):
        try:
            test_x[i, t] = word2idx(word)
        except:
            continue

print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)

test_x shape: (6198, 100)
test_y shape: (6198, 2)


In [37]:
train_y[:5]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [38]:
train_x[0]

array([10194,     0,  4642,    24,  1373,     0,   238,     0,  6409,
           0,     0,   198,     0,    59,   206,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [39]:
word_model.vectors[0]

array([ 1.0967804e+00, -2.2944486e+00,  1.9791678e+00,  3.4804371e-01,
        4.0753922e-01,  1.3286122e+00, -9.3368673e-01,  5.4947221e-01,
       -6.8077618e-01, -7.4963701e-01, -8.0936104e-02,  6.5788299e-02,
       -8.8564938e-01,  5.0793958e-01, -1.0864110e+00, -4.3317631e-01,
        2.0482888e-02, -5.7119979e-03, -1.0036458e+00,  3.1729680e-01,
        1.1956499e+00,  1.0685917e+00, -8.9309484e-01,  7.4419886e-01,
        4.4171312e-01, -2.0080043e-01, -2.6623638e+00,  1.7598321e-01,
       -2.0019765e+00, -5.6796205e-01, -2.0350738e-01, -7.3997623e-01,
        8.0226004e-01,  1.4174094e+00,  1.0990121e-01,  1.2313192e+00,
        1.6655500e+00, -2.6187131e-01,  1.7604357e+00, -3.1903556e-01,
        2.4266930e+00, -1.2458172e+00,  1.0440445e+00,  2.7978971e+00,
       -8.2489556e-01,  4.3603179e-01,  9.2311478e-01, -1.8360819e-01,
       -6.3582733e-02,  2.9669294e-01,  5.6388801e-01,  2.5535300e+00,
        2.3046949e+00, -1.8354168e+00, -6.8992484e-01, -1.3858901e+00,
      

In [40]:
# Создание своего Embedding

embedidding = Embedding(input_dim = vocab_size, output_dim = emdedding_size, weights = [pretrained_weights], trainable = False)

In [41]:
model = Sequential()
model.add(embedidding)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

2022-07-16 22:52:05.817030: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [43]:
tensorboard = TensorBoard(log_dir = './logs', write_graph = True, write_images = True)
early_stopping = EarlyStopping(monitor = 'val_loss')  


history = model.fit(train_x, train_y,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [tensorboard, early_stopping])

2022-07-16 22:52:06.738144: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-07-16 22:52:06.738166: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-07-16 22:52:06.738897: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-07-16 22:52:06.853627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/20
 2/26 [=>............................] - ETA: 3s - loss: 0.6910 - accuracy: 0.7305 

2022-07-16 22:52:07.706210: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-07-16 22:52:07.706225: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-07-16 22:52:07.836904: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-07-16 22:52:07.846693: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-07-16 22:52:07.861109: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_07_16_22_52_07
2022-07-16 22:52:07.861822: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2022_07_16_22_52_07/Ilyas-MBP.zyxel.box.trace.json.gz
2022-07-16 22:52:07.870335: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_07_16_22_52_07
2022-07-16 22:52:0

26/26 [==============================] - 5s 167ms/step - loss: 0.4984 - accuracy: 0.8070 - val_loss: 0.2918 - val_accuracy: 0.8749
Epoch 2/20
26/26 [==============================] - 3s 126ms/step - loss: 0.2804 - accuracy: 0.8828 - val_loss: 0.2736 - val_accuracy: 0.8784
Epoch 3/20
26/26 [==============================] - 3s 130ms/step - loss: 0.2484 - accuracy: 0.8991 - val_loss: 0.2641 - val_accuracy: 0.8846
Epoch 4/20
26/26 [==============================] - 4s 145ms/step - loss: 0.2381 - accuracy: 0.9065 - val_loss: 0.2603 - val_accuracy: 0.8784
Epoch 5/20
26/26 [==============================] - 4s 138ms/step - loss: 0.2224 - accuracy: 0.9120 - val_loss: 0.2552 - val_accuracy: 0.8784
Epoch 6/20
26/26 [==============================] - 4s 142ms/step - loss: 0.2141 - accuracy: 0.9183 - val_loss: 0.2520 - val_accuracy: 0.8880
Epoch 7/20
26/26 [==============================] - 4s 140ms/step - loss: 0.2109 - accuracy: 0.9264 - val_loss: 0.2546 - val_accuracy: 0.8825


In [44]:
results = model.predict(test_x, batch_size = batch_size, verbose = 1)

13/13 [==============================] - 1s 51ms/step


In [45]:
pred_y = np.argmax(results,axis = 1)

In [46]:
rusvectores_accuracy = accuracy_score(np.argmax(test_y,axis = 1), pred_y)
rusvectores_accuracy

0.8962568570506615

---

In [47]:
y_train = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)

In [48]:
model = Sequential()
model.add(Embedding(input_dim = max_words, output_dim = 128, input_length = max_sentence_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          128000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           49280     
_________________________________________________________________
activation_3 (Activation)    (None, 98, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [50]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [51]:
tensorboard = TensorBoard(log_dir = './logs', write_graph = True, write_images = True)
early_stopping = EarlyStopping(monitor = 'val_loss')  


history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [tensorboard, early_stopping])

Epoch 1/20


2022-07-16 22:52:34.146786: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-07-16 22:52:34.146802: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-07-16 22:52:34.146824: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


 2/26 [=>............................] - ETA: 2s - loss: 0.6719 - accuracy: 0.8135 

2022-07-16 22:52:34.920544: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-07-16 22:52:34.920562: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-07-16 22:52:35.032625: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-07-16 22:52:35.033722: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-07-16 22:52:35.034655: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_07_16_22_52_35
2022-07-16 22:52:35.037398: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2022_07_16_22_52_35/Ilyas-MBP.zyxel.box.trace.json.gz
2022-07-16 22:52:35.039646: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2022_07_16_22_52_35
2022-07-16 22:52:3

26/26 [==============================] - 4s 112ms/step - loss: 0.5766 - accuracy: 0.8128 - val_loss: 0.5027 - val_accuracy: 0.7996
Epoch 2/20
26/26 [==============================] - 3s 107ms/step - loss: 0.4795 - accuracy: 0.8138 - val_loss: 0.4967 - val_accuracy: 0.7996
Epoch 3/20
26/26 [==============================] - 3s 105ms/step - loss: 0.4780 - accuracy: 0.8122 - val_loss: 0.4953 - val_accuracy: 0.7996
Epoch 4/20
26/26 [==============================] - 3s 103ms/step - loss: 0.4815 - accuracy: 0.8092 - val_loss: 0.4934 - val_accuracy: 0.7996
Epoch 5/20
26/26 [==============================] - 3s 107ms/step - loss: 0.4723 - accuracy: 0.8135 - val_loss: 0.4961 - val_accuracy: 0.7996


In [52]:
results = model.predict(x_test, batch_size = batch_size, verbose = 1)

13/13 [==============================] - 0s 24ms/step


In [53]:
y_pred = np.argmax(results,axis = 1)

simple_accuracy = accuracy_score(df_test['Rating'].to_numpy(), y_pred)
simple_accuracy

0.8050984188447886

In [54]:
print('Результат:')
print(f'1. rusvectores: {rusvectores_accuracy:.4f}, 2. simple: {simple_accuracy:.4f}  ')

Результат:
1. rusvectores: 0.8963, 2. simple: 0.8051  


---